In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# hyperparameters

learning_rate = 1e-3
nb_epochs = 20
batch_size = 100

In [4]:
# MINST datasets

mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             train = True,
                             transform = transforms.ToTensor(),
                             download = True)

mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            train = False,
                            transform = transforms.ToTensor(),
                            download = True)

In [5]:
train_data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                                batch_size = batch_size,
                                                shuffle = True,
                                                drop_last = True)

In [6]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7 * 7 * 64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [7]:
model = CNN().to(device)

In [8]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# training
total_batch = len(train_data_loader)

for epoch in range(nb_epochs):
    avg_cost = 0
    
    for batch_idx, samples in enumerate(train_data_loader):
        X_train, Y_train = samples
        X_train = X_train.to(device)
        Y_train = Y_train.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X_train)
        
        cost = criterion(hypothesis, Y_train)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print("Epoch: {}/{} | batch_size: {}/{} | Cost: {:.6f}".format(
        epoch + 1, nb_epochs, 
        batch_idx + 1, len(train_data_loader),
        avg_cost))

Epoch: 1/20 | batch_size: 600/600 | Cost: 0.223968
Epoch: 2/20 | batch_size: 600/600 | Cost: 0.062216
Epoch: 3/20 | batch_size: 600/600 | Cost: 0.044878
Epoch: 4/20 | batch_size: 600/600 | Cost: 0.035580
Epoch: 5/20 | batch_size: 600/600 | Cost: 0.028992
Epoch: 6/20 | batch_size: 600/600 | Cost: 0.024863
Epoch: 7/20 | batch_size: 600/600 | Cost: 0.020801
Epoch: 8/20 | batch_size: 600/600 | Cost: 0.018079
Epoch: 9/20 | batch_size: 600/600 | Cost: 0.015115
Epoch: 10/20 | batch_size: 600/600 | Cost: 0.012702
Epoch: 11/20 | batch_size: 600/600 | Cost: 0.010369
Epoch: 12/20 | batch_size: 600/600 | Cost: 0.009820
Epoch: 13/20 | batch_size: 600/600 | Cost: 0.008443
Epoch: 14/20 | batch_size: 600/600 | Cost: 0.007610
Epoch: 15/20 | batch_size: 600/600 | Cost: 0.006074
Epoch: 16/20 | batch_size: 600/600 | Cost: 0.005611
Epoch: 17/20 | batch_size: 600/600 | Cost: 0.005240
Epoch: 18/20 | batch_size: 600/600 | Cost: 0.004408
Epoch: 19/20 | batch_size: 600/600 | Cost: 0.003945
Epoch: 20/20 | batch_

In [12]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

C:\Users\choih\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\choih\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9883999824523926
